# Task 3 - Epithelial to Mesenchymal (EMT) score from the Hallmark EMT or Epithelial to Mesenchymal Plasticity
This notebook analyses the Epithelial to Mesenchymal (EMT) score from the Hallmark EMT or Epithelial to Mesenchymal Plasticity score from [https://www.science.org/doi/10.1126/sciadv.abi7640](https://www.science.org/doi/10.1126/sciadv.abi7640) as well as HALLMARK.

This notebook is based on the single-cell best practices book (https://github.com/theislab/single-cell-best-practices/). For further explanations of the applied methods please refer to above source.

# Gene set enrichment and pathway analysis 

### Prepare and explore the data

In [ ]:
import os
os.environ['R_HOME'] = '/home/icb/till.richter/anaconda3/envs/scib-pipeline-R4.0/lib/R'

In [ ]:
import scanpy as sc
import anndata as ad
import numpy as np
import pandas as pd
import anndata as ad
import gdown
import anndata2ri
import rpy2
from rpy2.robjects import r
import random
import session_info

anndata2ri.activate()

In [ ]:
sc.settings.set_figure_params(dpi=200, frameon=False)
sc.set_figure_params(dpi=200)
sc.set_figure_params(figsize=(4, 4))

In [ ]:
%load_ext rpy2.ipython

In [ ]:
!python -m rpy2.situation

In [ ]:
%%R
suppressPackageStartupMessages({
    library(SingleCellExperiment)
    library(fgsea)
    library(AUCell)
    library(ggplot2)
})

In [ ]:
adata_read = sc.read_h5ad('../data/adata/processed_adata7.h5ad')
adata_read.obs

In [ ]:
adata = adata_read.copy()

In [ ]:
adata.layers['counts'] = adata.X

Let's keep a copy of the full dataset for the subsequent analyses before we subset on highly variable genes:

In [ ]:
adata_ = adata.copy()

In [ ]:
sc.pp.highly_variable_genes(
    adata_,
    n_top_genes=16000,
    flavor="seurat_v3",
    subset=True,
    layer="counts"
)

In [ ]:
adata.obs['group'] = adata.obs['louvain_r0.4'].astype('string')

In [ ]:
# find DE genes by t-test
adata.uns['log1p']["base"] = None
sc.tl.rank_genes_groups(adata, 'group', method='t-test', key_added = "t-test")

### Get the Pathway and map it to mm

In [ ]:
from pyorthomap import FindOrthologs 
hs2mm = FindOrthologs(
          host = 'http://www.ensembl.org',
          mart = 'ENSEMBL_MART_ENSEMBL',
          from_dataset = 'hsapiens_gene_ensembl',
          to_dataset = 'mmusculus_gene_ensembl',
          from_filters = 'hgnc_symbol',
          from_values = ['TP53', 'TERT'],
          to_attributes = ['external_gene_name'],
          to_homolog_attribute = 'mmusculus_homolog_ensembl_gene',
          from_gene_id_name = 'human_ensembl_gene_id',
          to_gene_id_name = 'mouse_ensembl_gene_id'
    )
    
hs2mm.map()

### S1 Pathway

In [ ]:
%%R
library(readxl)
my_pathways_s1=read_excel('../data/EMT_gene_sets/sciadv.abi7640_table_s1.xlsx')
my_pathways_s1=as.list(my_pathways_s1)

In [ ]:
df = pd.read_excel('../data/EMT_gene_sets/sciadv.abi7640_table_s1.xlsx') # can also index sheet by name or fetch all sheets
hs_pathway_genes_s1 = np.array(df['Conserved_EMP_Signature'].tolist())
len(hs_pathway_genes_s1)

In [ ]:
from pyorthomap import findOrthologsMmHs, findOrthologsHsMm
orthologies_hsmm = findOrthologsHsMm(from_filters = 'hgnc_symbol',
                  from_values = hs_pathway_genes_s1).map()
                  # from_values = [list(stats_df.index)]).map() # [stats.gene.names]).map()

In [ ]:
my_pathways_s1 = orthologies_hsmm['external_gene_name'].tolist()
my_pathways_s1[:10]

In [ ]:
%R -i my_pathways_s1

## S2 Pathway

In [ ]:
%%R
library(readxl)
my_pathways_s2=read_excel('../data/EMT_gene_sets/sciadv.abi7640_table_s2.xlsx')
my_pathways_s2=as.list(my_pathways_s2)
# my_pathways

In [ ]:
df = pd.read_excel('../data/EMT_gene_sets/sciadv.abi7640_table_s2.xlsx') # can also index sheet by name or fetch all sheets
hs_pathway_genes_s2 = np.array(df['Malignant_specific_EMP'].tolist())
len(hs_pathway_genes_s2)

In [ ]:
from pyorthomap import findOrthologsMmHs, findOrthologsHsMm
orthologies_hsmm = findOrthologsHsMm(from_filters = 'hgnc_symbol',
                  from_values = hs_pathway_genes_s2).map()
                  # from_values = [list(stats_df.index)]).map() # [stats.gene.names]).map()

In [ ]:
my_pathways_s2 = orthologies_hsmm['external_gene_name'].tolist()
my_pathways_s2[:10]

In [ ]:
%R -i my_pathways_s2

## Hallmark EMP Pathway

In [ ]:
%%R
all_pathways <- gmtPathways('../data/Hallmark/h.all.v7.5.1.symbols.gmt')
my_pathways_hallmark_ <- all_pathways['HALLMARK_EPITHELIAL_MESENCHYMAL_TRANSITION']

# Define the file name that will be deleted
fn <- '../data/EMT_gene_sets/hallmark_emt.csv'
# Check its existence
if (file.exists(fn)) {
  # Delete file if it exists
  file.remove(fn)
}

lapply(my_pathways_hallmark_, function(x) write.table( data.frame(x), fn, append=T, sep=','))
my_pathways_hallmark_

In [ ]:
hs_pathway_genes_hallmark = pd.read_csv('../data/EMT_gene_sets/hallmark_emt.csv')['x'].tolist()

In [ ]:
from pyorthomap import findOrthologsMmHs, findOrthologsHsMm
orthologies_hsmm = findOrthologsHsMm(from_filters = 'hgnc_symbol',
                  from_values = hs_pathway_genes_hallmark).map()
                  # from_values = [list(stats_df.index)]).map() # [stats.gene.names]).map()

In [ ]:
my_pathways_hallmark = orthologies_hsmm['external_gene_name'].tolist()
my_pathways_hallmark[:10]

In [ ]:
%R -i my_pathways_hallmark

### Cell-level pathway activity scoring using AUCell

In [ ]:
# Ignore this error. It occurs for some unknown reason, but adata is imported and you can continue to run the next cells
%R -i adata

In [ ]:
%%R
# Bioconductor 3.15 and R 4.2.0
# adata_aucell <- AUCell_run(adata_ , pathways)

cells_rankings <- AUCell_buildRankings(adata, plotStats=FALSE)
cells_AUC_s1 <- AUCell_calcAUC(my_pathways_s1, cells_rankings)
cells_AUC_s2 <- AUCell_calcAUC(my_pathways_s2, cells_rankings)
cells_AUC_hallmark <- AUCell_calcAUC(my_pathways_hallmark, cells_rankings)

In [ ]:
%%R
dim(cells_AUC_s1)

In [ ]:
%%R
dim(cells_AUC_s2)

In [ ]:
%%R
dim(cells_AUC_hallmark)

In [ ]:
%%R
# S1
aucell_scores_s1 <- data.frame(cells_AUC_s1@assays@data$AUC)
rownames(aucell_scores_s1) <- cells_AUC_s1@NAMES
colnames(aucell_scores_s1) <- colnames(adata)
# S2
aucell_scores_s2 <- data.frame(cells_AUC_s2@assays@data$AUC)
rownames(aucell_scores_s2) <- cells_AUC_s2@NAMES
colnames(aucell_scores_s2) <- colnames(adata)
# HALLMARK
aucell_scores_hallmark <- data.frame(cells_AUC_hallmark@assays@data$AUC)
rownames(aucell_scores_hallmark) <- cells_AUC_hallmark@NAMES
colnames(aucell_scores_hallmark) <- colnames(adata)

Export the rev_results back to python 

In [ ]:
%%R -o aucell_scores_s1
aucell_scores_s1 = aucell_scores_s1

In [ ]:
%%R -o aucell_scores_s2 
aucell_scores_s2 = aucell_scores_s2

In [ ]:
%%R -o aucell_scores_hallmark 
aucell_scores_hallmark = aucell_scores_hallmark

In [ ]:
aucell_scores_s1 = aucell_scores_s1.T
aucell_scores_s2 = aucell_scores_s2.T
aucell_scores_hallmark = aucell_scores_hallmark.T

We now add the scores for the interferon-related REACTOME pathways to the anndata object and annotate the activity level of these pathways in each of the cells on the UMAP:

In [ ]:
EMP_pathways = ['geneSet']

adata.obs['EMP Pathway S1 Cook et. al.'] = aucell_scores_s1[EMP_pathways]
adata.obs['EMP Pathway S2 Cook et. al.'] = aucell_scores_s2[EMP_pathways]
adata.obs['EMP Pathway\nHALLMARK_EPITHELIAL_MESENCHYMAL_TRANSITION'] = aucell_scores_hallmark[EMP_pathways]

In [ ]:
adata.obs['Cluster'] = adata.obs['louvain_r0.4']

Plot the scores on the umap

In [ ]:
sc.pl.umap(
    adata,
    color=["Cluster"],
    frameon=False,
    ncols=2,
    wspace = 0.4,
    save='/7_star-like/emp/UMAP_OnlyCluster.pdf'
)
os.rename("figures/umap/7_star-like/emp/UMAP_OnlyCluster.pdf", 
          "../rev_results/task3/7_star-like_emp_UMAP_OnlyCluster.pdf")

In [ ]:
sc.pl.umap(
    adata,
    color=["EMP Pathway S1 Cook et. al."],
    frameon=False,
    ncols=2,
    wspace = 0.4,
    save='/7_star-like/emp/UMAP_OnlyS1.pdf'
)
os.rename("figures/umap/7_star-like/emp/UMAP_OnlyS1.pdf", 
          "../rev_results/task3/7_star-like_emp_UMAP_OnlyS1.pdf")

In [ ]:
sc.pl.umap(
    adata,
    color=["EMP Pathway S2 Cook et. al."],
    frameon=False,
    ncols=2,
    wspace = 0.4,
    save='/7_star-like/emp/UMAP_OnlyS2.pdf'
)
os.rename("figures/umap/7_star-like/emp/UMAP_OnlyS2.pdf", 
          "../rev_results/task3/7_star-like_emp_UMAP_OnlyS2.pdf")

In [ ]:
sc.pl.umap(
    adata,
    color=["EMP Pathway\nHALLMARK_EPITHELIAL_MESENCHYMAL_TRANSITION"],
    frameon=False,
    ncols=2,
    wspace = 0.4,
    save='/7_star-like/emp/UMAP_OnlyHallmark.pdf'
)
os.rename("figures/umap/7_star-like/emp/UMAP_OnlyHallmark.pdf", 
          "../rev_results/task3/7_star-like_emp_UMAP_OnlyHallmark.pdf")

### Violin

In [ ]:
sc.pl.violin(adata, 
             keys=["EMP Pathway S1 Cook et. al."],
             groupby='Cluster', 
             rotation=90,
             xlabel='Cluster',
             inner="quartile",
             wspace = 0.4,
             save='/7_star-like/emp_violin_counts_OnlyS1.pdf'
            )
os.rename("figures/violin/7_star-like/emp_violin_counts_OnlyS1.pdf", 
          "../rev_results/task3/7_star-like_emp_violin_counts_OnlyS1.pdf")

In [ ]:
sc.pl.violin(adata, 
             keys=["EMP Pathway S2 Cook et. al."],
             groupby='Cluster', 
             rotation=90,
             xlabel='Cluster',
             inner="quartile",
             wspace = 0.4,
             save='/7_star-like/emp_violin_counts_OnlyS2.pdf'
            )
os.rename("figures/violin/7_star-like/emp_violin_counts_OnlyS2.pdf", 
          "../rev_results/task3/7_star-like_emp_violin_counts_OnlyS2.pdf")

In [ ]:
sc.pl.violin(adata, 
             keys=["EMP Pathway\nHALLMARK_EPITHELIAL_MESENCHYMAL_TRANSITION"],
             groupby='Cluster', 
             rotation=90,
             xlabel='Cluster',
             inner="quartile",
             wspace = 0.4,
             save='/7_star-like/emp_violin_counts_OnlyHallmark.pdf'
            )
os.rename("figures/violin/7_star-like/emp_violin_counts_OnlyHallmark.pdf", 
          "../rev_results/task3/7_star-like_emp_violin_counts_OnlyHallmark.pdf")